In [1]:
# Import required libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.optimizers import Adam
import keras_tuner as kt
from sklearn.model_selection import TimeSeriesSplit

In [4]:
# Load Dataset
file_path = "preprocessed_dataset.csv"  # Change this to your file
data = pd.read_csv(file_path, parse_dates=["date_and_time"], index_col="date_and_time")

In [5]:
# Selecting Features and Target
features = ["PM2.5", "O3", "TEMPERATURE", "PRESSURE", "DEWPOINT", "RAIN"]
target = "PM2.5"  # Predicting PM2.5 levels

In [6]:
# Scaling Data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data[features])

In [7]:
# Splitting into Train & Test Sets (80-20)
train_size = int(len(scaled_data) * 0.8)
train_data, test_data = scaled_data[:train_size], scaled_data[train_size:]

In [8]:

# Function to Create Sequences
def create_sequences(data, time_steps=10):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i : i + time_steps])
        y.append(data[i + time_steps, 0])  # Target is PM2.5
    return np.array(X), np.array(y)

time_steps = 10
X_train, y_train = create_sequences(train_data, time_steps)
X_test, y_test = create_sequences(test_data, time_steps)

In [9]:
# Define LSTM Model for Hyperparameter Tuning
def build_lstm_model(hp):
    model = Sequential()
    model.add(LSTM(units=hp.Int('units', min_value=32, max_value=128, step=32), 
                   return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dropout(hp.Float('dropout', min_value=0.1, max_value=0.5, step=0.1)))
    model.add(LSTM(units=hp.Int('units_2', min_value=32, max_value=128, step=32)))
    model.add(Dense(1))  # Output Layer
    model.compile(loss="mse", optimizer=Adam(learning_rate=hp.Choice("learning_rate", [0.001, 0.0005, 0.0001])))
    return model

In [14]:
# Hyperparameter Tuning
tuner = kt.RandomSearch(
    build_lstm_model, objective="val_loss", max_trials=5, executions_per_trial=2, directory="lstm_tuning"
)

tuner.search(X_train, y_train, epochs=50, validation_data=(X_test, y_test), batch_size=8)

Trial 5 Complete [00h 04m 50s]
val_loss: 0.0007800710445735604

Best val_loss So Far: 0.0007737351406831294
Total elapsed time: 00h 15m 29s


In [15]:
# Get Best Hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

In [16]:
# Train Best Model
best_model = tuner.hypermodel.build(best_hps)
history = best_model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=1)

Epoch 1/50
191/191 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0022 - val_loss: 0.0012
Epoch 2/50
191/191 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0013 - val_loss: 0.0012
Epoch 3/50
191/191 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0014 - val_loss: 0.0012
Epoch 4/50
191/191 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0017 - val_loss: 0.0011
Epoch 5/50
191/191 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0013 - val_loss: 0.0011
Epoch 6/50
191/191 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0012 - val_loss: 0.0010
Epoch 7/50
191/191 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0012 - val_loss: 9.9913e-04
Epoch 8/50
191/191 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0010 - val_loss: 9.6657e-04
Epoch 9/50
191/191 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 9.1863e-04 - val_loss: 9.7699e-04
Epoch 10/50
191/191 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 9.4031e-04 - val_loss: 0.0011
Epoch 11/50
191/191 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 9.6815e-04 - val_loss: 9.3217e-04
Epoch 12/50
191/191 ━━━━

In [17]:
# Predictions
y_pred = best_model.predict(X_test)

48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


In [18]:
# Rescale Predictions & Actual Values
y_pred_rescaled = scaler.inverse_transform(np.concatenate((y_pred, np.zeros((y_pred.shape[0], len(features) - 1))), axis=1))[:, 0]
y_test_rescaled = scaler.inverse_transform(np.concatenate((y_test.reshape(-1, 1), np.zeros((y_test.shape[0], len(features) - 1))), axis=1))[:, 0]

In [19]:
# Model Evaluation
mse = mean_squared_error(y_test_rescaled, y_pred_rescaled)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test_rescaled, y_pred_rescaled)
r2 = r2_score(y_test_rescaled, y_pred_rescaled)

print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"R² Score: {r2}")

Mean Squared Error (MSE): 0.000781929010993196
Root Mean Squared Error (RMSE): 0.027962993598561584
Mean Absolute Error (MAE): 0.010616907827157773
R² Score: 0.4579437279251447
